In [1]:
# Import necessary libraries
from datetime import datetime
from IPython.display import clear_output

import ee
import geemap


In [2]:
# Authenticate if required
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

### Cloud Clearing


In [3]:
# Cloud clearing params

CLOUD_FILTER = 60 # This value is used because in some areas are permanently masked due to false positive
# CLD_PRB_THRESH = 40
# CLD_PRB_THRESH = 50
CLD_PRB_THRESH = 60
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100

In [4]:
### Dataset Download Functions

def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    # s2_sr_s2HR_filtered_cloudless = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))


def get_s2SR(aoi= ee.Geometry.Point(103.851959, 1.290270)):
    """
    Returns image collection of COPERNICUS/S2_SR, filtered by the AOI\n
    Params:\n
        aoi: Area of Interest <ee.Geometry.Point>\n
    """
    # Import and filter S2 SR.
    # s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [5]:
### Dataset filtering functions

def filterCollection(collection, params, filter_type):
    """
    Filters a collection by the date, can be individual or range, see filter types below:\n
    Y_S: Filter by custom years, pass in list of years in "YYYY" format\n
    M_S: Filter by custom months, pass in list of months in "MM" format\n
    Y_R: Filter by custom years, pass in list of tuples, [(start_incl, end_excl)] of years in "YYYY" format\n
    M_R: Filter by custom months, pass in list of tuples, [(start_incl, end_excl)] of months in "MM" format\n
    """

    col = None
    
    # Filtering based on params
    if (filter_type == "Y_S"):
        for year in params:
            if col is None:
                col = collection.filter(ee.Filter.calendarRange(year, year, "year"))
            else:
                col = col.merge(col.filter(ee.Filter.calendarRange(year, year, "year")))

    elif filter_type == "M_S":
        for month in params:
            if col is None:
                col = collection.filter(ee.Filter.calendarRange(month, month, "month"))
            else:
                col = col.merge(col.filter(ee.Filter.calendarRange(month, month, "month")))
            
    elif filter_type == "Y_R":
        for years in params:
            if col is None:
                col = collection.filter(ee.Filter.calendarRange(years[0], years[1], "year"))
            else:
                col = col.merge(col.filter(ee.Filter.calendarRange(years[0], years[1], "year")))
    elif filter_type == "M_R":
        for months in params:
            if col is None:
                col = collection.filter(ee.Filter.calendarRange(months[0], months[1], "month"))
            else:
                col = col.merge(col.filter(ee.Filter.calendarRange(months[0], months[1], "month")))
    else:
        print(f"Wrong Filter Type: {filter_type}")
        raise ValueError

    return col

# Date retrieval function
def getDates(collection, fmt= "%Y/%m/%d %H:%M"):
    """
    Gets the dates of images of an ImageCollection in a list, format is specified by the datetime strftime format
    """
    # Retrieve the dates 
    date_lst = collection.aggregate_array("system:time_start")

    # Download to local
    date_lst = date_lst.getInfo()

    # Parse into datetime
    date_lst = list(map(lambda x: datetime.fromtimestamp(x / 1e3).strftime(fmt), date_lst))
    
    return date_lst



In [6]:
### Definition of functions for cloud masking

def maskS2clouds(image):
    """
    Returns a sentinel-2 image with the clouds masked 
    """
    qa60 = image.select("QA60")

    # The bit mask for QA60 as specified in the documentation
    cloudBitMask = 1 << 10 # Indicates opaque clouds present
    cirrusBitMask = 1 << 11 # Indicates cirrus clouds present

    # mask = ((qa60 & cloudBitMask) == 0) and ((qa60 & cirrusBitMask) == 0)
    mask = qa60.bitwiseAnd(cloudBitMask).eq(0).And(qa60.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)


def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')))

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)


In [7]:
# Setting of parameters
CITY_COORDS = [129.3145, 36.0030]
CITY = ee.Geometry.Point(CITY_COORDS) # Coord format: E, N 

# Singapore
sg_lon = 103.851959
sg_lat = 1.290270
CITY_COORDS = [sg_lon, sg_lat]
CITY = ee.Geometry.Point(sg_lon, sg_lat)

# Years of interest
YEARS = [2020]


In [8]:
# Initialise and render map

Map = geemap.Map()
Map.setCenter(*CITY_COORDS, 13)
Map

Map(center=[1.29027, 103.851959], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [9]:
# Get the image collection and filter the dates
s2HR = get_s2SR(CITY) # City is the geometry point object of the city

# Filters the collection by the selected year
s2HR_filtered = filterCollection(s2HR, YEARS, "Y_S")
s2HR_filtered = filterCollection(s2HR_filtered, [1], "M_S")

# Removes the pixels that are covered by clouds for each image in the collection
s2HR_filtered_cloudless = s2HR_filtered.map(add_cld_shdw_mask).map(apply_cld_shdw_mask)

In [10]:
# Information of the collection
# The bands in the image
print(f"Bands: {s2HR_filtered_cloudless.first().bandNames().getInfo()}")
print(f"Number of Images in Collection: {s2HR_filtered_cloudless.size().getInfo()}")

Bands: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']
Number of Images in Collection: 3


In [11]:
# Retrieves the median
s2HR_median = s2HR_filtered_cloudless.median()

In [12]:
# Render origial with clouds
# Map.addLayer(s2HR.median(), {"min": 0, "max": 2500, "gamma": 1.1, "bands": ["B4", "B3", "B2"]}, "RGB")

# Renders the new layer
Map.addLayer(s2HR_median, {"min": 0, "max": 2500, "gamma": 1.1, "bands": ["B4", "B3", "B2"]}, "RBG_cloudless")

In [13]:
# Retrieve the dates 
date_lst = getDates(s2HR_filtered_cloudless)

print(f"Total Entries (filtered): {len(date_lst)}")
# print(f"Total Entries (original): {s2_sr_cld_col_eval.size().getInfo()}")
print("Preview: ")
print(*date_lst[:5], sep= "\n")
print("......")
print(*date_lst[-5:], sep= "\n")

# Prints all
# print(*date_lst, sep= "\n")



Total Entries (filtered): 3
Preview: 
2020/01/01 12:37
2020/01/06 12:37
2020/01/26 12:37
......
2020/01/01 12:37
2020/01/06 12:37
2020/01/26 12:37


In [14]:
# Testing of filter function

s2HR = get_s2SR()

#Filter by individual years
print("Individual Years")
s2HR_filtered_test = filterCollection(s2HR, [2020, 2022], filter_type="Y_S")
print(getDates(s2HR_filtered_test))
print()

# Filter by individual months
print("Individual Months")
s2HR_filtered_test = filterCollection(s2HR, [3, 7], "M_S")
print(getDates(s2HR_filtered_test))
print()

# Filter by range of years
print("Range Years")
s2HR_filtered_test = filterCollection(s2HR, [(2017, 2020)], "Y_R")
print(getDates(s2HR_filtered_test))
print()

# Filter by range of months
print("Range Months")
s2HR_filtered_test = filterCollection(s2HR, [(1, 3), (11, 13)], "M_R")
print(getDates(s2HR_filtered_test))
print()

# Filter by range of years, specific months
print("Range Years, Individual Months")
s2HR_filtered_test = filterCollection(s2HR, [(2020, 2023)], "Y_R")
s2HR_filtered_test = filterCollection(s2HR_filtered_test, [1, 3, 9], "M_S")
print(getDates(s2HR_filtered_test))
print()


Individual Years
['2020/01/01 12:37', '2020/01/06 12:37', '2020/01/26 12:37', '2020/02/05 12:37', '2020/02/15 12:37', '2020/02/20 12:37', '2020/02/25 12:37', '2020/03/01 12:37', '2020/03/11 12:37', '2020/03/16 12:37', '2020/03/21 12:37', '2020/03/26 12:37', '2020/03/31 12:37', '2020/04/05 12:37', '2020/04/10 12:37', '2020/04/15 12:37', '2020/04/20 12:37', '2020/05/05 12:37', '2020/05/15 12:37', '2020/05/25 12:37', '2020/05/30 12:37', '2020/06/09 12:37', '2020/06/19 12:37', '2020/07/04 12:37', '2020/08/28 12:37', '2020/09/02 12:37', '2020/09/17 12:37', '2020/09/22 12:37', '2020/10/02 12:37', '2020/10/27 12:37', '2020/11/16 12:37', '2020/12/01 12:37', '2020/12/26 12:37']

Individual Months
['2019/03/02 12:37', '2019/03/12 12:37', '2019/03/17 12:37', '2019/03/22 12:37', '2019/03/27 12:37', '2020/03/01 12:37', '2020/03/11 12:37', '2020/03/16 12:37', '2020/03/21 12:37', '2020/03/26 12:37', '2020/03/31 12:37', '2021/03/01 12:37', '2021/03/06 12:37', '2021/03/11 12:37', '2021/03/16 12:37', '2

In [15]:
# Test cell

# Properties of ImageCollection and Image
print("### Properties of Image Collection")
print(sorted(s2HR.propertyNames().getInfo()))
print()

print("### Properties of Image")
print(sorted(s2HR.first().propertyNames().getInfo()))
# print(s2HR.get("title").getInfo())
# print(s2HR.get("system:id").getInfo())

### Properties of Image Collection
['date_range', 'description', 'keywords', 'period', 'product_tags', 'provider', 'provider_url', 'sample', 'source_tags', 'system:id', 'system:version', 'system:visualization_0_bands', 'system:visualization_0_max', 'system:visualization_0_min', 'system:visualization_0_name', 'tags', 'thumb', 'title', 'type_name', 'visualization_0_bands', 'visualization_0_max', 'visualization_0_min', 'visualization_0_name']

### Properties of Image
['AOT_RETRIEVAL_ACCURACY', 'CLOUDY_PIXEL_PERCENTAGE', 'CLOUD_COVERAGE_ASSESSMENT', 'CLOUD_SHADOW_PERCENTAGE', 'DARK_FEATURES_PERCENTAGE', 'DATASTRIP_ID', 'DATATAKE_IDENTIFIER', 'DATATAKE_TYPE', 'DEGRADED_MSI_DATA_PERCENTAGE', 'FORMAT_CORRECTNESS', 'GENERAL_QUALITY', 'GENERATION_TIME', 'GEOMETRIC_QUALITY', 'GRANULE_ID', 'HIGH_PROBA_CLOUDS_PERCENTAGE', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B1', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B10', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B11', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B12', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B2', 

In [16]:
# print(s2HR.get("system:id").getInfo())
# print(s2HR.first().get("system:id").getInfo())
# print(sorted(s2HR.toList(s2HR.size().getInfo()).get(1).getInfo()["properties"]["system:index"]))
# print(s2HR.toList(s2HR.size().getInfo()).get(1).getInfo()["bands"][0]["id"])
# print((s2HR.toList(s2HR.size().getInfo()).get(1).getInfo()["id"]))

# Retrieve data of image
image = s2HR.toList(s2HR.size().getInfo()).get(1)
print(type(image))
image = ee.Image(image)
print(type(image))

img_dict = image.getInfo()
band_sample_id = img_dict["bands"][0]["id"]
id_sample_name = img_dict["id"]
property_sample_time = img_dict["properties"]["system:time_start"]
property_sample_time = datetime.fromtimestamp(property_sample_time / 1e3).strftime("%Y/%m/%d_%H:%M")
projection = image.select(band_sample_id).projection().getInfo()

print(band_sample_id)
print(id_sample_name)
print(property_sample_time)
print(projection)

<class 'ee.computedobject.ComputedObject'>
<class 'ee.image.Image'>
B1
COPERNICUS/S2_SR/20190106T032129_20190106T033238_T48NUG
2019/01/06_12:37
{'type': 'Projection', 'crs': 'EPSG:32648', 'transform': [60, 0, 300000, 0, -60, 200040]}


In [17]:
s2HR.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32648',
   'crs_transform': [60, 0, 300000, 0, -60, 200040]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32648',
   'crs_transform': [10, 0, 300000, 0, -10, 200040]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32648',
   'crs_transform': [10, 0, 300000, 0, -10, 200040]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32648',
   'crs_transform': [10, 0, 300000, 0, -10, 200040]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,


# Export Images

In [18]:
### Function Definitions for exporting images

def boundingRectangle(aoi, width, height, projection):
    """
    Returns ee.Geometry.Rectangle centered on AOI with width and height in metres\n
    aoi: ee.Geometry.Point\n
    width, height: Dimensions in metres\n
    projection: Type of project used, eg.'EPSG:32648'
    """

    # Create bounding rectangle
    # https://gis.stackexchange.com/questions/363706/area-and-dimensions-of-ee-geometry-rectangle
    xRadius = width/2
    yRadius = height/2
    pointLatLon = aoi
    pointMeters = pointLatLon.transform(projection, 0.001)
    # coords = pointLatLon.coordinates()
    coords = pointMeters.coordinates()
    minX = ee.Number(coords.get(0)).subtract(xRadius)
    minY = ee.Number(coords.get(1)).subtract(yRadius)
    maxX = ee.Number(coords.get(0)).add(xRadius)
    maxY = ee.Number(coords.get(1)).add(yRadius)
    # rect = ee.Geometry.Rectangle([minX, minY, maxX, maxY])
    rect = ee.Geometry.Rectangle([minX, minY, maxX, maxY], projection, False)
    return rect

def create_bounding_box(point: ee.Geometry.Point, diameter: float):
    intermediate_circle = point.buffer(diameter / 2.0, 0.0)
    return intermediate_circle.bounds()

def exportImageToDrive(image, aoi, bound, dir, projection, dt= None, prefix= "S2HR"):
    """
    Takes in an image and exports it to the specific directory in drive\n
    Files are named as <prefix>_<YYYYMMDD_HHMM> by default\n
    image: ee.Image, assumed filtered before input\n
    aoi: Geometry.Point(Long, Lat) of the area of interest\n
    bound: The (width, height) of the bound centered on the AOI in metres\n
    If ee.Geometry.Rectangle is provided, bound will not be re-calculated within the function\n
    dir: Drive directory\n
    projection: The projection information (dict) of the image, should contain "crs" and "transform"\n
    dt: Date string corresponding to image in <YYYY/MM/DD HH:MM> format. \n
    Will be automatically generated if blank.\n
    prefix: The prefix of the files\n

    Returns:\n
        task: task object for the uploading of the file

    ### WIP ###
    Remove need to recalulate bounding box for each image\n
    Auto selection of highest resolution band for projection\n
    Get the bands first \n
    Allow for hard user preference \n

    """

    # Retrieve data of image
    # if projection is None:
    #     band_sample_id = img_dict["bands"][0]["id"]
    #     # id_sample_name = img_dict["id"]
    #     # The projection will also determine the scale/resolution of the output image, so selection of bands is significant
    #     for band in ["B2", "B3", "B4"]: # These bands are the usually the RGB bands and are of the highest resolution
    #         try:  
    #             projection = image.select(band).projection().getInfo()
    #             break
    #         except:
    #             continue
    #     if projection is None: # If none of the bands specified above works, use the first band
    #         projection = image.select(band_sample_id).projection().getInfo()
                 
    if dt is None:
        img_dict = image.getInfo()
        try:
            dt = datetime.fromtimestamp(img_dict["properties"]["system:time_start"] / 1e3).strftime("%Y%m%d_%H%M")
        except:
            dt = "UNKNOWN_DT"

    # Create bounding rectangle if not passed
    if not isinstance(bound, ee.Geometry):
        bounding_box = boundingRectangle(aoi, bound[0], bound[1], projection["crs"])
    else:
        bounding_box = bound
    # bounding_box = create_bounding_box(aoi, bound[0])

    # geemap.ee_export_image(
    #     image, filename= dir, region=bounding_box, file_per_band=False, scale= 90
    #     )

    task = ee.batch.Export.image.toDrive(image,
                                    description=f"{prefix}_{dt}",
                                    folder=dir,
                                    crs=projection['crs'],
                                    crsTransform=projection['transform'],
                                    region=bounding_box,
                                    fileFormat="GeoTIFF",
                                    maxPixels=10000000000000,
                                    )

    try:                         
        task.start()
    except:
        try:
            ee.Initialize()
        except:
            ee.Authenticate()
            ee.Initialize()
        finally:
            task.start()
        

    return task
        
    
def exportCollectionToDrive(collection, aoi, bound, dir, date_lst= None, projection= None, projection_bands= None, prefix= "S2HR"):
    """
    Takes in an image collection and exports it to the specific directory in drive\n
    collection: ee.ImageCollection, assumed filtered before input\n
    aoi: Geometry.Point(Long, Lat) of the area of interest\n
    bound: The (width, height) of the bound centered on the AOI in metres\n
    dir: Drive directory\n
    date_lst: A list of dates corresponding to each image in the collection. Will be automatically generated if blank.
    projection: The projection information of the image, should contain crs and transform\n
    projection_bands: A list of bands from which to choose the projection information from\n
    prefix: The prefix of the files

    """
    ### Iterate through the collection and export each image individually

    # Convert image collection to list of images
    col_size = collection.size().getInfo()
    col_lst = collection.toList(col_size)

    # Converts each element from ee.ComputedObject to ee.Image
    img_lst = []
    for i in range(col_size):
        img_lst.append(ee.Image(col_lst.get(i)))

    # Create lst to store tasks
    task_lst = []

    # Retrieve necessary data

    if date_lst is None:
        date_lst = getDates(collection, "%Y%m%d_%H%M")
    if projection is None:
        if projection_bands is None:
            raise ValueError("Projection bands must be provided if projection is not provided. Please refer to documentation for available bands.")
        col_bands = collection.first().bandNames().getInfo()
        sel_band = None
        for band in projection_bands:
            if band in col_bands:
                sel_band = band
                break
        if sel_band is None:
            raise KeyError("Band not found, check for case and typos.")

        projection = collection.first().select(sel_band).projection().getInfo()
    
    # Calculate the bounds to prevent repeated calculation
    bounding_box = boundingRectangle(aoi, bound[0], bound[1], projection["crs"])

    # Iterate and export
    for img_i in range(col_size):
        task_lst.append(exportImageToDrive(
                                            image= img_lst[img_i], 
                                            aoi= aoi, 
                                            bound= bounding_box, 
                                            dir= dir, 
                                            dt= date_lst[img_i], 
                                            projection= projection, 
                                            prefix= prefix,
                                            )
                                            )

    return task_lst

    
    
    

In [19]:
# Retrieves the median
s2HR_mean = s2HR_filtered_cloudless.mean()
s2HR_mean

In [20]:
# Retrieve projection information from the first in the collection
projection = s2HR_filtered_cloudless.first().select("B2").projection().getInfo()

In [30]:
# Sanity export 1
task = exportImageToDrive(s2HR_filtered_cloudless.first(), CITY, (10000, 10000), "data_test", projection, prefix= "first")

In [31]:
# valid = True
# prev_s = None
# print(task.status())
# while valid:
#     d = task.status()
#     clear_output(wait=True)
#     # print(d)
#     print(d["state"])
#     print(round((d["update_timestamp_ms"] - d["start_timestamp_ms"])/1000, 1))
#     valid = not (d["state"] == "COMPLETED" or d["state"] == "FAILED")
# print(d)



In [32]:
# Test export 1
task = exportImageToDrive(s2HR_filtered_cloudless.median(), CITY, (10000, 10000), "data_test", projection= projection, prefix= "median")

In [33]:
# valid = True
# prev_s = None
# print(task.status())
# while valid:
#     d = task.status()
#     clear_output(wait=True)
#     # print(d)
#     print(d["state"])
#     print(round((d["update_timestamp_ms"] - d["start_timestamp_ms"])/1000, 1))
#     valid = not (d["state"] == "COMPLETED" or d["state"] == "FAILED")
# print(d)



In [34]:
# Test export 2
task = exportImageToDrive(s2HR_filtered_cloudless.mean(), CITY, (10000, 10000), "data_test", projection= projection, prefix= "mean")

In [35]:
# valid = True
# prev_s = None
# print(task.status())
# while valid:
#     d = task.status()
#     clear_output(wait=True)
#     # print(d)
#     print(d["state"])
#     print(round((d["update_timestamp_ms"] - d["start_timestamp_ms"])/1000, 1))
#     valid = not (d["state"] == "COMPLETED" or d["state"] == "FAILED")
# print(d)



In [36]:
# Test Export 3

tasks = exportCollectionToDrive(s2HR_filtered_cloudless, CITY, (10000, 10000), "data_test", projection_bands= ["B2"], prefix= "Col")